In [ ]:
pip install functions-framework

In [ ]:
import functions_framework
from pytz import timezone
from datetime import datetime, timedelta
import pandas as pd
import requests
import sqlalchemy
import pymysql
import os

In [ ]:
@functions_framework.http
def flights_pushing(request):
    schema = "city_scooter_analysis"  # The name of your database
    host = "104.199.15.9"
    user = "root"
    password = "Insert_your_SQL_root_password" # Replace with your actual variable or hardcode for testing
    port = 3306

    # Create the connection string for SQLAlchemy
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    # Read the cities table into a DataFrame
    input_airport_df = pd.read_sql("airports", con=connection_string)
    input_icao_list = input_airport_df['icao']

    # Preparing the querystring by selecting API parameters. 
    querystring = {"withLeg":"True",
                "direction":"Arrival",
                "withCancelled":"False",
                "withCodeshared":"True",
                "withCargo":"False",
                "withPrivate":"False",
                "withLocation":"False"
                }

    headers = 	{
                "x-rapidapi-key": 'Insert_your_API_key', #  Replace with your actual API key. Using the string without imports helps debug. In Cloud use Google secrets
                "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
                }

    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = (today + timedelta(days=1))
    times = [["00:00","11:59"], # Maximum time span for the API is 12 hours. So run twice for selected time intervals to receive full day's data
            ["12:00","23:59"]]

    flight_num = []
    departure_icao = []
    arrival_icao = []
    arrival_time = []
    retrieval_time = []


    retrieval_timestamp = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

    for airport in input_icao_list:
        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

            flights_response = requests.get(url, headers=headers, params=querystring)
            try:
                flights_json = flights_response.json()
            except requests.JSONDecodeError:
                continue
            for flight in flights_json['arrivals']:
                flight_num.append(flight['number'])
                try:
                    departure_icao.append(flight['departure']['airport']['icao'])
                except:
                    departure_icao.append('')
                arrival_icao.append(airport)
                arrival_time.append(flight['arrival']['scheduledTime']['local'])
                retrieval_time.append(retrieval_timestamp)
    flights_df = pd.DataFrame({"flight_num": flight_num,
                                        "departure_icao": departure_icao,
                                        "arrival_icao": arrival_icao,
                                        "arrival_time": arrival_time,
                                        "retrieval_time": retrieval_time
                                            })
    flights_df["arrival_time"] = flights_df["arrival_time"].str[:-6] # Removing info to convert to datetime format
    flights_df["arrival_time"] = pd.to_datetime(flights_df["arrival_time"])
    flights_df["retrieval_time"] = pd.to_datetime(flights_df["retrieval_time"])

    # This is only for testing without the table join
    flights_df.to_sql('flights', if_exists="append", con=connection_string, index=False) 

    return "Data has been sent to the table"

In [ ]:
from flask import Request
import json

request_data = {}
request = Request.from_values(data=json.dumps(request_data))

# Call the function
response = flights_pushing(request)
print(response)